In [1]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data

data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [3]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

# Calculating $\delta$, $\theta_1$, $\xi_{jt}$

Where $\theta_1$ = $(\alpha,\beta)$

In [4]:
#set up useful global variables 
NMKTS = data['Market_ID'].nunique()
NPLANS = data['Plan_ID'].nunique()
NOBS = data['Plan_ID'].count()
NSIM = 20


#set up variables for testing purposes
v = np.random.normal(size=(NSIM,3,NOBS)) #initilize v
theta2 = np.array([3,2,1]) # initialize theta2 for testing purposes
delta = np.ones(NOBS)*(-2)

#print global variables
print NMKTS,NPLANS,NOBS

600 16 3300


In [5]:
def cal_s(data, delta, theta2, v):
    """Calculate market share
    Calculates individual choice probability first, then take sum"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO']]).transpose()
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1,3300))
    delta  = np.tile( delta  ,(NSIM,1))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() ) 
    
    #sum up between markets
    sim_exp['mkt_id'] = data['Market_ID']
    sum_exp = sim_exp.groupby('mkt_id').sum() 
    sum_exp = pd.merge(data.copy()[['Market_ID']], sum_exp, 
                       right_index=True, left_on = 'Market_ID')
    
    #format so I can broadcast
    sim_exp = np.array(sim_exp).transpose()[:-1]
    sum_exp = np.array(sum_exp).transpose()[1:] + 1
    
    shares = (1./NSIM)*(sim_exp/sum_exp).sum(axis=0)
    return shares


s = cal_s(data, delta, theta2, v)

print s.mean(), s.min(), s.max(), s.shape

0.14198643275571607 0.00994763616352982 0.44135146443458667 (3300,)


In [6]:
def cal_delta(v, data, theta2, error = 1e-6, maxiter = 500):
    """Calculate mean utility via contraction mapping"""

    niter = 0
    
    #initialize loop parameters
    delta = np.zeros(NOBS)
    s = cal_s(data, delta, theta2, v)
    diff = np.log(data['Inside Good Share']) - np.log(s)
    
    
    while ((abs(diff).max() > error) 
           and (abs(diff).mean() > error) 
           and niter < maxiter):
        
        s = cal_s(data, delta, theta2, v)
        diff = np.log(data['Inside Good Share']) - np.log(s)

        if np.isnan(diff).sum():
            raise Exception('nan in diffs')
            
        delta += diff
        niter += 1

    print 'contraction mapping finished in %s iterations'%niter
    return delta


delta = cal_delta(v, data, theta2)
print delta.shape
print delta.mean()

contraction mapping finished in 142 iterations
(3300,)
-1.40861569885


In [15]:
#initialize theta1 i.e. alpha and beta
theta1 = np.array([3,2,1,-2])

def cal_xi(x, delta, theta1, theta2):
    """Calculate xi with F.O.C"""
    xi = delta - np.matmul(np.array(x),theta1)
    return  xi

xi = cal_xi( x, delta,  theta1, theta2)
print xi.max(), xi.min(), xi.mean()

2.6523886388302307 -6.9354322179925205 -1.26972057836


In [46]:
def cal_theta1(X1,Z,delta, theta1_init, theta2):
    """  calculate theta 1 using FOCs (X1'Z T Z'X )^-1 X1'Z T Z' delta """
    
    theta1 = theta1_init
    
    #estimation requires two steps
    for i in range(2):
        
        X1 , Z = np.array(x), np.array(Z) 
        
        #on first step, use consistent approximation of T
        if i==0:
            T =  np.linalg.inv( Z.transpose().dot(Z) )
            
        #on second step use estimated T using xi
        if i==1:
            xi = cal_xi(x, delta, theta1, theta2)
            xi =np.array([xi]).transpose()
            T =  np.linalg.inv( Z.transpose().dot(xi).dot(xi.transpose()).dot(Z) )
        
        #build up to main equation
        XtZ = X1.transpose().dot(Z)
        ZtX = Z.transpose().dot(X1)
        
        first_exp = np.linalg.inv( XtZ.dot(T).dot(ZtX))
        second_exp = XtZ.dot(T).dot(Z.transpose()).dot(delta)
        theta1 = first_exp.dot(second_exp)
      
    return theta1


print cal_theta1(x,hausman_instr,delta,theta1, theta2)

[ 6.44456807  3.34557032  1.6705884  -4.32421496]


# Calculating  $\theta_2$


Use GMM over $\theta_2$, the non-linear parameters. 

In [72]:
#first estimate using logit
class BLP(GMM):
    
    def __init__(self, data, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        self.data = data
        super(BLP, self).__init__(*args, **kwds)
        
        
    def momcond(self, params):
        #unwrap stuff
        data = self.data
        #print x
        print data

        print data['Market ID']
        
        xi = 0
        #g = instr * xi
        
        return g

    
#momcond is E(z'xi(x,theta2)) = 0    

#print data

#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model = BLP(data, y, x, hausman_instr)

result = model.fit(beta_init, maxiter=2, optim_method='nm', 
                   wargs=dict(centered=False))

print(result.summary())

TypeError: 'PandasData' object has no attribute '__getitem__'